In [2]:
import time
import random
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from numpy import genfromtxt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Process protein (AA) data

In [3]:
metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Ecoli1936metafiles/metadata_final.csv")

### Load data

In [4]:
aafold = '0' # '1', '2'

In [57]:
kmer_matrix_VT1_fi = np.load('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_mat_VT1.npy') # save numpy array
selected_features_fi = np.load('/data/hoan/amromics/prediction/data/kmer_Fold1_mat_VT'+aafold+'_features.npy') # save numpy array

In [58]:
kmer_matrix_VT1_fi.shape, selected_features_fi.shape

((600, 2935739), (2935739,))

In [36]:
# df1 = pd.DataFrame(data = kmer_matrix_VT1_f1, columns = selected_features_f1)
# df2 = pd.DataFrame(data = kmer_matrix_VT1_f2, columns = selected_features_f2)

In [61]:
# df_full = pd.concat([df0,df1,df2], axis=0, ignore_index=True) # fail: it's too big

In [37]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2

In [37]:
if aafold == '0':
    start_idx = 0; end_idx = 600
elif aafold == '1':
    start_idx = 600; end_idx = 1200
else:
    start_idx = 1200; end_idx = metadata_panta.shape[0]

In [ ]:
mutual_mat = []
for idx in range(2, 14):
    # y_class = metadata_panta.iloc[:600,idx].values
    y_class = metadata_panta.iloc[start_idx:end_idx, idx].values
    print(metadata_panta.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = kmer_matrix_VT1_fi[nonenan_index, ]
    y_new = y[nonenan_index].astype(int)
    if len(y_new) > 10:
        scores, pvalue = chi2(pa_matrix_new, y_new)
        mutual_mat.append(scores)
mutual_mat = np.array(mutual_mat)

In [44]:
mutual_mat_mean = mutual_mat.mean(axis=0)

In [46]:
top_features = np.argsort(mutual_mat_mean)[::-1][:100000]
kmer_matrix_VT_top_features = kmer_matrix_VT1_fi[:,top_features]
selected_features_top = selected_features_fi[top_features]

(600, 100000)

In [50]:
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_mat_VT1_top_features.npy', kmer_matrix_VT_top_features) # save numpy array
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_mat_VT1_features_top_features.npy', selected_features_f0_top) # save numpy array

In [50]:
stop

### Run aafold for 0, 1, 2 and then concat 3 datasets together

In [63]:
data0 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold0_mat_VT1_top_features.npy') # save numpy array
feature0 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold0_mat_VT1_features_top_features.npy') # save numpy array
df0 = pd.DataFrame(data = data0, columns = feature0)
#
data1 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold1_mat_VT1_top_features.npy') # save numpy array
feature1 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold1_mat_VT1_features_top_features.npy') # save numpy array
df1 = pd.DataFrame(data = data1, columns = feature1)
#
data2 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold2_mat_VT1_top_features.npy') # save numpy array
feature2 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold2_mat_VT1_features_top_features.npy') # save numpy array
df2 = pd.DataFrame(data = data2, columns = feature2)

In [66]:
df0.shape, df1.shape, df2.shape

((600, 100000), (600, 100000), (453, 100000))

In [67]:
df_full = pd.concat([df0,df1,df2], axis=0, ignore_index=True) 

In [69]:
df_full = df_full.fillna(0)

In [71]:
snp_mat = df_full.values

In [75]:
np.save('/data/hoan/amromics/prediction/data/kmer_full_mat_VT1_AA.npy', snp_mat) # save numpy array